In [ ]:
!pip install rapidfuzz
from google.colab import drive

!pip install python-Levenshtein
import Levenshtein

drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.2 MB 6.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 174 kB 7.1 MB/s 
Mounted at /content/drive


In [ ]:
import pandas as pd
from scipy.spatial.distance import pdist
from rapidfuzz import process, fuzz
import numpy as np

In [ ]:
# Get data, create usable format
book_pivot = pd.read_csv('/content/drive/MyDrive/book_pivot.csv', index_col=[0])
book_meta = pd.read_csv('/content/drive/MyDrive/book_meta.csv', index_col=[0])

# Create frame for storing distances
dist_frame = pd.DataFrame(index=book_meta.index, data=book_meta[['title']])

In [ ]:
# Function to search movie name, returns index in database
def search(name, index):
    dist = []

    for t in book_meta['title']:
        dist.append(Levenshtein.distance(t.lower(), name.lower(), weights=(100, 1, 10)))

    df = pd.DataFrame(book_meta[['title']])
    df['dist'] = dist
    selection = df.sort_values('dist', ascending=True).head(10).reset_index()

    display(selection[['title']])

    print('\n\nSelected:')

    display(pd.DataFrame(selection.iloc[index]).T)

    return selection['item_id'][index]

In [ ]:
# Search for movie 1. Store distances to other movies

base_id = search('Game of thrones', 1)

dist_frame['Dist_1'] = ((book_pivot - book_pivot.loc[base_id]) ** 2).sum(axis=1).to_list()

,title
0,"A Game of Thrones: The Graphic Novel, Vol. 1"
1,"A Game of Thrones (A Song of Ice and Fire, #1)"
2,An Abundance of Katherines
3,"Shades of Earth (Across the Universe, #3)"
4,The Smell of Other People's Houses
5,"Games of the Heart (The 'Burg, #4)"
6,"The Wolf Gift (The Wolf Gift Chronicles, #1)"
7,"What's Left of Me (The Hybrid Chronicles, #1)"
8,"Tower of Dawn (Throne of Glass, #6)"
9,"Stalk Me (The Keatyn Chronicles, #1)"




Selected:


,item_id,title,dist
1,1466917,"A Game of Thrones (A Song of Ice and Fire, #1)",31


In [ ]:
# Search for movie 2. Store distances to other movies

base_id = search('twilight', 0)
dist_frame['Dist_2'] = ((book_pivot - book_pivot.loc[base_id]) ** 2).sum(axis=1).to_list()

,title
0,The Twilight Wife
1,"Eclipse (Twilight, #3)"
2,Twelfth Night
3,"Twilight (Twilight, #1)"
4,"New Moon (Twilight, #2)"
5,This Raging Light
6,"Twilight (The Mediator, #6)"
7,"Breaking Dawn (Twilight, #4)"
8,West with the Night
9,"Midnight Sun (Twilight, #1.5)"




Selected:


,item_id,title,dist
0,49874249,The Twilight Wife,9


In [ ]:
# Store weighted average

w_1 = .5
w_2 = 1 - w_1

dist_frame['Dist_3'] = (dist_frame['Dist_1'] ** 2 * w_1) + (dist_frame['Dist_2'] ** 2 * w_2)

In [ ]:
# Search based on one of the three types of distance (Dist_1, Dist_2, or Dist_3)

dist_frame.sort_values('Dist_3').head(20)

,title,Dist_1,Dist_2,Dist_3
item_id,,,,
19258458,"The Witch of Duva (The Grisha, #0.5)",779.353076,698.488685,547638.829832
15093325,"Shadow and Bone (The Grisha, #1)",637.420272,874.847466,585831.346527
2715267,"The Goose Girl (The Books of Bayern, #1)",692.489737,836.472441,589614.089693
21474467,"The Retribution of Mara Dyer (Mara Dyer, #3)",842.066421,704.866690,602956.453922
1069505,"The Thief (The Queen's Thief, #1)",727.706591,824.224296,604451.286326
14668834,This Dark Endeavor (The Apprenticeship of Vict...,775.611561,795.156695,616923.731634
6528619,"The Demon King (Seven Realms, #1)",727.391428,852.603647,628015.633559
15727095,"For You (The 'Burg, #1)",790.260397,795.772274,628882.503866
14926833,"The Emperor's Edge (The Emperor's Edge, #1)",802.372435,785.283599,630235.927355
